In [11]:
!poetry update -q

In [12]:
from dotenv import load_dotenv

from configurations.configuration import Configuration
from configurations.cypher_chat_openai import CypherChatOpenAIConfiguration
from configurations.neo4j import Neo4jConfiguration
from configurations.openai_embeddings import OpenAIEmbeddingsConfiguration

load_dotenv()

configurations = Configuration.load('../configuration.yaml')
neo4j = Neo4jConfiguration.grab(configurations)
openai_embeddings = OpenAIEmbeddingsConfiguration.grab(configurations)
cypher_chat_openai = CypherChatOpenAIConfiguration.grab(configurations)

In [13]:
f = 0
t = 5

In [14]:
import pyarrow.parquet as pq

table = pq.read_table('train-00000-of-00001.parquet', memory_map=True)
strings: list[str] = []
imported_rows: list[str] = []
for index, cell in enumerate(table[4][f:t]):
    imported_rows.append(table[1][f + index].as_py())
    strings.append(cell.as_py())

In [16]:
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from sro import SRO

llm = ChatOpenAI(
    model_name='gpt-4o',
    temperature=0
)

prompt_template = """
Analyze the following text and extract all the facts in a structured way. Provide the facts in the format:

Subject: SUBJECT OF FACT
Relationship: TYPE OF RELATIONSHIP
Object: FACT OBJECT

Examples of facts:
Subject: Alice, Relationship: works at, Object: Acme Corp.
Subject: New York, Relationship: is, Object: city

Text: "{text}"
"""

prompt = PromptTemplate(
    input_variables=['text'],
    template=prompt_template
)

entity_extraction_chain = LLMChain(llm=llm, prompt=prompt)

facts: list[str] = []
for string in strings[f:t]:
    # splitlines = string.splitlines()
    # print(len(splitlines))
    # for index, line in enumerate(splitlines):
    #     print(index)
    #     facts.append(entity_extraction_chain.run(line))
    facts += entity_extraction_chain.run(string).split('\n\n')
facts

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
sros = list(filter(lambda sro: sro.is_full(), [SRO(fact) for fact in facts]))
for sro in sros:
    print(sro)

In [ ]:
from langchain_community.graphs import Neo4jGraph

query = ''
for index, sro in enumerate(sros):
    query += f"""
    MERGE (a{index}:Entity {{name: '{sro.subject}'}})
    MERGE (b{index}:Entity {{name: '{sro.object}'}})
    MERGE (a{index})-[:{sro.relation.replace(' ', '_').replace('-', '_').upper()}]->(b{index})"""
graph = Neo4jGraph(**neo4j.dict())
graph.query(query)

with open('imported_links.txt', 'w') as file:
    file.writelines([f"{row}\n" for row in imported_rows])